# test

In [2]:
import sys
import os
import numpy as np
import pandas as pd

# load reference data from a file
ref_path = '../temp/reference.csv'
ref_data = pd.read_csv(ref_path)
ref_data


,id,problem,answer
0,057f8a,Three airline companies operate flights from D...,79
1,192e23,Fred and George take part in a tennis tourname...,250
2,1acac0,Triangle $ABC$ has side length $AB = 120$ and ...,180
3,1fce4b,Find the three-digit number $n$ such that writ...,143
4,349493,"We call a sequence $a_1, a_2, \ldots$ of non-n...",3
5,480182,"Let $ABC$ be a triangle with $BC=108$, $CA=126...",751
6,71beb6,"For a positive integer $n$, let $S(n)$ denote ...",891
7,88c219,"For positive integers $x_1,\ldots, x_n$ define...",810
8,a1d40b,The Fibonacci numbers are defined as follows: ...,201
9,bbd91e,Alice writes all positive integers from $1$ to...,902


In [4]:
# show a problem
import textwrap

wrapped_text = textwrap.fill(ref_data.iloc[2, 1], width=50)
print(wrapped_text)

Triangle $ABC$ has side length $AB = 120$ and
circumradius $R = 100$. Let $D$ be the foot of the
perpendicular from $C$ to the line $AB$. What is
the greatest possible length of segment $CD$?


# Qwen/Qwen2.5-Math-7B-Instruct

<https://huggingface.co/Qwen/Qwen2.5-Math-7B-Instruct>

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer

model_name = "Qwen/Qwen2.5-Math-7B-Instruct"
device = "cuda" # the device to load the model onto

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype="auto",
    device_map="auto"
)
tokenizer = AutoTokenizer.from_pretrained(model_name)

prompt = "Find the value of $x$ that satisfies the equation $4x+5 = 6x+7$."

# CoT
messages = [
    {"role": "system", "content": "Please reason step by step, and put your final answer within \\boxed{}."},
    {"role": "user", "content": prompt}
]

# TIR
messages = [
    {"role": "system", "content": "Please integrate natural language reasoning with programs to solve the problem above, and put your final answer within \\boxed{}."},
    {"role": "user", "content": prompt}
]

text = tokenizer.apply_chat_template(
    messages,
    tokenize=False,
    add_generation_prompt=True
)
model_inputs = tokenizer([text], return_tensors="pt").to(device)

generated_ids = model.generate(
    **model_inputs,
    max_new_tokens=512
)
generated_ids = [
    output_ids[len(input_ids):] for input_ids, output_ids in zip(model_inputs.input_ids, generated_ids)
]

response = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)[0]
